## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Qaanaaq,GL,77.4840,-69.3632,-13.50,86,99,5.99,overcast clouds
1,1,Hobart,AU,-42.8794,147.3294,61.00,63,40,14.97,scattered clouds
2,2,Ranghulu,CN,46.6500,124.8667,49.17,65,1,16.73,clear sky
3,3,Tura,IN,25.5198,90.2201,69.76,15,0,2.57,clear sky
4,4,Wewak,PG,-3.5534,143.6268,80.51,84,100,4.23,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Wewak,PG,-3.5534,143.6268,80.51,84,100,4.23,moderate rain
11,11,Hambantota,LK,6.1241,81.1185,77.05,78,85,4.21,overcast clouds
18,18,Saint-Philippe,RE,-21.3585,55.7679,75.20,83,0,5.75,light rain
19,19,Soyo,AO,-6.1349,12.3689,80.11,82,86,9.01,overcast clouds
20,20,Vaini,TO,-21.2000,-175.2000,86.00,74,75,6.91,broken clouds
29,29,Rikitea,PF,-23.1203,-134.9692,79.72,69,1,17.27,clear sky
31,31,Del Rio,US,29.3627,-100.8968,93.20,10,1,6.91,clear sky
34,34,Cap Malheureux,MU,-19.9842,57.6142,77.00,82,79,1.99,broken clouds
37,37,Butaritari,KI,3.0707,172.7902,81.19,79,92,13.06,light rain
39,39,Ambulu,ID,-8.3450,113.6058,78.06,84,66,2.48,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                197
City                   197
Country                197
Lat                    197
Lng                    197
Max Temp               197
Humidity               197
Cloudiness             197
Wind Speed             197
Current Description    197
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                197
City                   197
Country                197
Lat                    197
Lng                    197
Max Temp               197
Humidity               197
Cloudiness             197
Wind Speed             197
Current Description    197
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Wewak,PG,80.51,moderate rain,-3.5534,143.6268,
11,Hambantota,LK,77.05,overcast clouds,6.1241,81.1185,
18,Saint-Philippe,RE,75.20,light rain,-21.3585,55.7679,
19,Soyo,AO,80.11,overcast clouds,-6.1349,12.3689,
20,Vaini,TO,86.00,broken clouds,-21.2000,-175.2000,
29,Rikitea,PF,79.72,clear sky,-23.1203,-134.9692,
31,Del Rio,US,93.20,clear sky,29.3627,-100.8968,
34,Cap Malheureux,MU,77.00,broken clouds,-19.9842,57.6142,
37,Butaritari,KI,81.19,light rain,3.0707,172.7902,
39,Ambulu,ID,78.06,broken clouds,-8.3450,113.6058,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Wewak,PG,80.51,moderate rain,-3.5534,143.6268,In Wewak Boutique Hotel
11,Hambantota,LK,77.05,overcast clouds,6.1241,81.1185,Bungalow 63
18,Saint-Philippe,RE,75.20,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
19,Soyo,AO,80.11,overcast clouds,-6.1349,12.3689,Hotel Mpampa
20,Vaini,TO,86.00,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
29,Rikitea,PF,79.72,clear sky,-23.1203,-134.9692,Pension Maro'i
31,Del Rio,US,93.20,clear sky,29.3627,-100.8968,Holiday Inn Express & Suites del Rio
34,Cap Malheureux,MU,77.00,broken clouds,-19.9842,57.6142,Pereybere Hotel & Spa
37,Butaritari,KI,81.19,light rain,3.0707,172.7902,Isles Sunset Lodge
39,Ambulu,ID,78.06,broken clouds,-8.3450,113.6058,Hotel Ambulu Jember


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [34]:
# 11a. Add a marker layer for each city to the map. 
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(25.0, -50.0), zoom_level=2.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))